In [2]:
# Note: need to install psycopg2 from source if using in production environment
# https://www.psycopg.org/docs/install.html
# %pip install sqlglot sqlvalidator sqlalchemy psycopg2-binary sqlfluff mysql-connector-python pyodbc google-cloud-bigquery
# %pip install pyflakes pylint parso flake8 mypy ruff
# %pip install docker

In [ ]:
from importlib import reload
import sql_parsers
reload(sql_parsers)

import python_parsers
reload(python_parsers)

In [1]:
import docker
import pandas as pd
import time

from functools import partial

from python_parsers import *
from sql_parsers import *

In [2]:
sql_queries = pd.read_csv('/mnt/foundation-shared/nina_xu_gretel_ai/datasets/sql_queries_w_dialect_1000.csv')
sql_queries_googlesql = pd.read_csv('/mnt/foundation-shared/nina_xu_gretel_ai/datasets/sql_queries_googlesql_200.csv')
sql_queries = pd.concat([sql_queries, sql_queries_googlesql], ignore_index=True)
python_typscript_codes = pd.read_csv('/mnt/foundation-shared/nina_xu_gretel_ai/datasets/python_typescript_codes.csv')
python_codes = pd.read_json('/mnt/foundation-shared/nina_xu_gretel_ai/datasets/text_to_python_v1.json')

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 100)

# Give each row a unique db name because otherwise BigQuery struggles with the same db name
sql_queries['id_tmp'] = sql_queries.index
sql_queries['db_name'] = sql_queries.apply(lambda x: f"db_{x.id_tmp}", axis=1)

# Basic cleaning. BigQuery errors out if there are newlines like 'CREATE\nTABLE'
sql_queries['SQL Query'] = sql_queries['SQL Query'].apply(
    lambda x: x.replace('CREATE\nTABLE', 'CREATE TABLE').replace('CREATE\nVIEW', 'CREATE VIEW').replace('INSERT\nINTO', 'INSERT INTO'))
sql_queries['Context'] = sql_queries['Context'].apply(
    lambda x: x.replace('CREATE\nTABLE', 'CREATE TABLE').replace('CREATE\nVIEW', 'CREATE VIEW').replace('INSERT\nINTO', 'INSERT INTO'))


# SQL Code Validation

In [ ]:
sql_queries.head(1)
print(sql_queries.Dialect.value_counts())

sql_queries.tail(1)

In [ ]:
sql_queries.Complexity.value_counts()

In [ ]:
"""
Have a PostgreSQL database running in a Docker container. In command line, run the following commands:
# Grant access to non-root users so that the python client will work
> sudo groupadd docker
> sudo usermod -aG docker $USER
> newgrp docker

> docker pull postgres
> docker run --name my-postgres \
  -e POSTGRES_USER=myuser \
  -e POSTGRES_PASSWORD=mypassword \
  -e POSTGRES_DB=mydatabase \
  -p 5433:5432 \
  -d postgres

"""
client = docker.from_env()

# List all running containers
containers = client.containers.list(all=False)
# Get the postgres container
postgres_container = client.containers.get('my-postgres')
# Get container's gateway, not that it's not the "IPAddress" field
postgres_container_gateway = postgres_container.attrs['NetworkSettings']['Gateway']
print(postgres_container_gateway)

postgres_db_creds = {
        "host": postgres_container_gateway,
        "port": 5433, # the default port is 5432, but that was already in use for me
        "user": "myuser",
        "password": "mypassword",
        "dbname": "my-postgres",
    }

In [ ]:
"""
Have a MySQL database running in a Docker container. In command line, run the following commands:
> docker pull mysql
> docker run --name my-mysql \
  -e MYSQL_ROOT_PASSWORD=myrootpassword \
  -d mysql
"""

mysql_container = client.containers.get('my-mysql')
mysql_container_ip = mysql_container.attrs['NetworkSettings']['IPAddress']
print(mysql_container_ip)

mysql_db_creds = {
    "host": mysql_container_ip,
    "port": 3306, # default port for mysql
    "user": "root",
    "password": "myrootpassword",
}

In [ ]:
"""
Have a Microsoft SQL Server database running in a Docker container. In command line, run the following commands:
$ docker pull mcr.microsoft.com/mssql/server
$ docker run --name my-sqlserver \
  -e 'ACCEPT_EULA=Y' -e 'MSSQL_SA_PASSWORD=myRoot(!)Password' \
  -p 1433:1433 \
  -d mcr.microsoft.com/mssql/server

$ sudo apt install unixodbc-dev

Install the SQL Server command-line tool (sqlcmd) inside the container:
$ docker exec -it --user root my-sqlserver bash
# apt-get update
# apt-get install -y mssql-tools unixodbc-dev
"""
          
sqlserver_container = client.containers.get('my-sqlserver')
sqlserver_container_ip = sqlserver_container.attrs['NetworkSettings']['IPAddress']
print(sqlserver_container_ip)

sqlserver_db_creds = {
    "host": sqlserver_container_ip,
    "port": 1433, # default port for sql server,
    "user": "sa",
    "password": "myRoot(!)Password",
}

In [100]:
"""
Have a BigQuery emulator running in a Docker container. The official BigQuery image requires authentication 
to Google Cloud and would actually interact with BigQuery. In command line, run the following commands:

$ docker pull ghcr.io/goccy/bigquery-emulator:latest
$ docker run -it -p 9050:9050 ghcr.io/goccy/bigquery-emulator:latest --project=test-project

Note: if running the same SQL queries again, kill the container and start a fresh one because 
the deleting dataset functionality was not working as expected.
"""

biquery_db_creds = {
     "port": 9050,
     "project": "test-project",
}

In [101]:
#  Apply different SQL validators to the SQL queries
def is_valid_query_and_schema(row, func):
    query_check = func(row['SQL Query'])
    schema_check = func(row['Context'])
    is_valid_schema = schema_check[0]
    is_valid_query = query_check[0]
    is_valid_sql = is_valid_schema and is_valid_query
    error_messages = f"***Schema error: {schema_check[1]}" if not is_valid_schema else ''
    error_messages += f"***Query error: {query_check[1]}" if not is_valid_query else ''
    return is_valid_sql, is_valid_schema, is_valid_query, error_messages

def is_valid_query_and_schema_with_sqlfluff(row):
    dialect_map = {
        'SQLite': 'sqlite',
        'PostgreSQL': 'postgres',
        'MySQL': 'mysql',
        'SQL Server': 'tsql',
        'GoogleSQL': 'bigquery',
        'Oracle': 'oracle',
    }
    if 'Oracle' in row['Dialect']:
        dialect = 'oracle'
    else:
        dialect = dialect_map.get(row['Dialect'], 'ansi')
    query_check = SimpleSqlValidator.is_valid_sql_with_sqlfluff(row['SQL Query'], dialect)
    schema_check = SimpleSqlValidator.is_valid_sql_with_sqlfluff(row['Context'], dialect)
    is_valid_schema = schema_check[0]
    is_valid_query = query_check[0]
    is_valid_sql = is_valid_schema and is_valid_query
    error_messages = f"***Schema error: {schema_check[1]}" if not is_valid_schema else ''
    error_messages += f"***Query error: {query_check[1]}" if not is_valid_query else ''
    return is_valid_sql, is_valid_schema, is_valid_query, error_messages

def check_query_and_schema_separately(sql_queries, method):
    start_time = time.time()
    functions_to_apply = {
        'sqlglot': partial(is_valid_query_and_schema, func=SimpleSqlValidator.is_valid_sql_with_sqlglot),
        'sqlquery': partial(is_valid_query_and_schema, func=SimpleSqlValidator.is_valid_sql_with_sqlquery),
        'sqlfluff': is_valid_query_and_schema_with_sqlfluff,
    }

    result = sql_queries.apply(functions_to_apply[method], axis=1).apply(list)
    sql_queries[f'is_valid_sql_with_{method}'] = result.apply(lambda x: x[0])
    sql_queries[f'is_valid_schema_with_{method}'] = result.apply(lambda x: x[1])
    sql_queries[f'is_valid_query_with_{method}'] = result.apply(lambda x: x[2])
    sql_queries[f'error_msgs_{method}'] = result.apply(lambda x: x[3])

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"{method} check executed in {elapsed_time:.2f} seconds")

    return sql_queries


def check_query_against_schema(row, dialect):

    validator_classes = {
        'SQLite': SqliteValidator,
        'PostgreSQL': PostgresqlValidator,
        'MySQL': MysqlValidator,
        'SQL Server': SqlserverValidator,
        'GoogleSQL': GooglesqlValidator,
    }

    kwargs_postgres = {
        'domain': row['Topic'],
        'db_creds': postgres_db_creds,
    }
    kwargs_mysql = {
        'domain': row['Topic'],
        'db_creds': mysql_db_creds,
        'mysql_container': mysql_container,
    }
    kwargs_sqlserver = {
        'domain': row['Topic'],
        'db_creds': sqlserver_db_creds,
        'sqlserver_container': sqlserver_container,
    }
    kwargs_bigquery = {
        'domain': row['db_name'],
        'db_creds': biquery_db_creds,
    }

    all_kwargs = {
        'SQLite': {},
        'PostgreSQL': kwargs_postgres,
        'MySQL': kwargs_mysql,
        'SQL Server': kwargs_sqlserver,
        'GoogleSQL': kwargs_bigquery
    }

    dialect_name = dialect.lower().replace(' ', '')

    if row['Dialect'] == dialect:
        result = validator_classes[dialect].is_valid_sql(
            row['SQL Query'], row['Context'], **all_kwargs[dialect]
            )
    else:
        result = None, None
    
    row[f'is_valid_{dialect_name}'] = result[0]
    row[f'error_msg_{dialect_name}'] = result[1]
    
    return row

def apply_check_query_against_schema(sql_queries, dialect):
    start_time = time.time()
    sql_queries = sql_queries.apply(check_query_against_schema, dialect=dialect, axis=1)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"{dialect} check executed in {elapsed_time:.2f} seconds")
    return sql_queries


In [ ]:
sql_queries = check_query_and_schema_separately(sql_queries, 'sqlfluff')
# sql_queries = check_query_and_schema_separately(sql_queries, 'sqlglot')
# sql_queries = check_query_and_schema_separately(sql_queries, 'sqlquery')

# sql_queries = apply_check_query_against_schema(sql_queries, 'SQLite')
# sql_queries = apply_check_query_against_schema(sql_queries, 'PostgreSQL')
# sql_queries = apply_check_query_against_schema(sql_queries, 'MySQL')
# sql_queries = apply_check_query_against_schema(sql_queries, 'SQL Server')
# sql_queries = apply_check_query_against_schema(sql_queries, 'GoogleSQL')

print(sql_queries.is_valid_sql_with_sqlglot.value_counts(normalize=True))
print(sql_queries.is_valid_sql_with_sqlquery.value_counts(normalize=True))
print(sql_queries.is_valid_sql_with_sqlfluff.value_counts(normalize=True))

print(sql_queries.is_valid_sqlite.value_counts(normalize=True))
print(sql_queries.is_valid_postgresql.value_counts(normalize=True))
print(sql_queries.is_valid_mysql.value_counts(normalize=True))
print(sql_queries.is_valid_sqlserver.value_counts(normalize=True))
print(sql_queries.is_valid_googlesql.value_counts(normalize=True))


In [ ]:
def get_googlesql_error_categories(error_msg):
    if not error_msg:
        return None
    googlesql_error_categories = ['Type not found', 'Syntax error', 'Foreign keys are not supported', 'does not support']
    for category in googlesql_error_categories:
        if category.lower() in error_msg.lower():
            return category


sql_queries['googlesql_error_category'] = sql_queries['error_msg_googlesql'].apply(get_googlesql_error_categories)

remaining = sql_queries[(sql_queries.is_valid_googlesql == False) & (sql_queries.googlesql_error_category.isnull())][['SQL Query', 'Context', 'error_msg_googlesql']]
print(sql_queries.googlesql_error_category.value_counts())
print(remaining.count())
# remaining.head()

In [3]:

# Coalesce the results of the five dialects into a single column
sql_queries['is_valid_sql'] = sql_queries['is_valid_sqlite'].fillna(
    sql_queries['is_valid_mysql'].fillna(
        sql_queries['is_valid_postgresql'].fillna(
            sql_queries['is_valid_sqlserver'].fillna(
                sql_queries['is_valid_googlesql']
            )
        )
    )
)

sql_queries['error_msg_sql'] = sql_queries['error_msg_sqlite'].fillna(
    sql_queries['error_msg_mysql'].fillna(
        sql_queries['error_msg_postgresql'].fillna(
            sql_queries['error_msg_sqlserver'].fillna(
                sql_queries['error_msg_googlesql']
            )
        )
    )
)


In [2]:
# sql_queries.to_csv('/mnt/foundation-shared/nina_xu_gretel_ai/datasets/sqlqueries_1200_validated_092524.csv', index=False)
# sql_queries = pd.read_csv('/mnt/foundation-shared/nina_xu_gretel_ai/datasets/sqlqueries_1200_validated_092524.csv')

In [ ]:
dialects = ['SQLite', 'PostgreSQL', 'MySQL', 'SQL Server', 'GoogleSQL']
for dialect in dialects:
    print(f"\n***{dialect}***")
    print(sql_queries[sql_queries['Dialect'] == dialect].is_valid_sql_with_sqlglot.value_counts(normalize=True))
    print(sql_queries[sql_queries['Dialect'] == dialect].is_valid_sql_with_sqlquery.value_counts(normalize=True))
    print(sql_queries[sql_queries['Dialect'] == dialect].is_valid_sql_with_sqlfluff.value_counts(normalize=True))

In [ ]:
methods = ['sqlglot', 'sqlquery', 'sqlfluff']
for dialect in dialects:
    print(f"\n***{dialect}***")
    for method in methods:
        print(f"***{method}***")
        print(sql_queries[sql_queries['Dialect'] == dialect][f'is_valid_sql_with_{method}'].value_counts(normalize=True))
        print(sql_queries[sql_queries['Dialect'] == dialect][f'is_valid_schema_with_{method}'].value_counts(normalize=True))
        print(sql_queries[sql_queries['Dialect'] == dialect][f'is_valid_query_with_{method}'].value_counts(normalize=True))


In [ ]:
# Check if the query is valid with both sqlglot and sqlfluff
# SQLQuery is proven to be useless so not counting it in the aggregate
sql_queries['is_valid_sql_aggregate'] = sql_queries[['is_valid_sql_with_sqlglot', 'is_valid_sql_with_sqlfluff']].all(axis=1)
for dialect in dialects:
    print(f"***{dialect}***")
    print(sql_queries[sql_queries['Dialect'] == dialect].is_valid_sql_aggregate.value_counts(normalize=True))

In [ ]:
# What are the differences between checking against schema and validating the query separately from schema?
for dialect in dialects:
    print(f"\n***{dialect}***")
    dialect_name = dialect.lower().replace(' ', '')
    df = sql_queries[sql_queries['Dialect'] == dialect]
    print(pd.crosstab(df[f'is_valid_{dialect_name}'], df['is_valid_sql_with_sqlfluff']))

In [ ]:
dialect = dialects[4]
dialect_name = dialect.lower().replace(' ', '')
print(dialect)
df = sql_queries[(sql_queries['Dialect'] == dialect) & 
                 ((sql_queries['is_valid_sql_with_sqlfluff'] == False) & 
                  (sql_queries[f'is_valid_{dialect_name}'] == True))]
df[['SQL Query', 'Context', f'error_msg_{dialect_name}', 'error_msgs_sqlfluff', 'error_msgs_sqlquery', 'error_msgs_sqlglot']].head()

In [ ]:
print(sql_queries['Context'].loc[5])

# Python Code Validation

In [74]:
python_check_methods = {
    # 'compile': is_valid_python_with_complie,
    # 'ast': is_valid_python_with_ast,
    # 'pyflakes': is_valid_python_with_pyflakes,
    # 'parso': is_valid_python_with_parso,
    # 'mypy': is_valid_python_with_mypy,
    'ruff': is_valid_python_with_ruff,
    # 'ruff_extensive': is_valid_python_with_ruff,
    # 'ruff_pyflakes': is_valid_python_with_ruff,
    # 'pylint': is_valid_python_with_pylint,
}

def check_python_code_with_method(df, method='compile', **kwargs):
    start_time = time.time()

    func = python_check_methods[method]
    df[f'check_{method}'] = df['code'].apply(func, **kwargs)
    df[f'is_valid_python_with_{method}'] = df[f'check_{method}'].apply(lambda x: x[0])
    df[f'{method}_error'] = df[f'check_{method}'].apply(lambda x: x[1])

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"\n{method} check executed in {elapsed_time:.2f} seconds")
    
    return df

In [ ]:

for method in python_check_methods.keys():
    python_codes = check_python_code_with_method(python_codes, method)

# python_codes = check_python_code_with_method(python_codes, 'ruff_extensive', level='warning')
# ruff_pyflakes_args = {
#     'level': 'custom',
#     'ruff_rules': ['F'],
# }
# python_codes = check_python_code_with_method(python_codes, 'ruff_pyflakes', **ruff_pyflakes_args)

for method in python_check_methods.keys():
    print(python_codes[f'is_valid_python_with_{method}'].value_counts())

In [ ]:
if 'pylint_error' in python_codes.columns:
    python_codes['pylint_score'] = python_codes['pylint_error'].apply(lambda x: x['score'] if x else None)
    python_codes['pylint_severity'] = python_codes['pylint_error'].apply(lambda x: x['severity'] if x else None)
    python_codes['pylint_messages'] = python_codes['pylint_error'].apply(lambda x: x['messages'] if x else None)

print(python_codes.pylint_severity.value_counts())
print(python_codes.groupby('pylint_severity')['pylint_score'].mean())

print(pd.crosstab(python_codes['is_valid_python_with_ruff'], python_codes['is_valid_python_with_pylint']))
    

In [ ]:
print(python_codes['code'].loc[648])

In [61]:
# method = 'pylint'
# python_codes[python_codes[f'is_valid_python_with_{method}'] == False][['code', 
# 'compile_error', '' f'{method}_error']].head(10)
# python_codes[python_codes['is_valid_python_with_pylint'] == False][['code', 'pyflakes_error', 'ruff_error', 'pylint_score', 'pylint_severity', 'pylint_messages']].head(10)

In [ ]:
python_codes[(python_codes.is_valid_python_with_ruff == False) & (python_codes.is_valid_python_with_pylint == True)][['code', 'ruff_error' ,'pylint_messages']].head(20)

In [ ]:

def get_error_category(error: str, error_categories: list) -> str:
    
    for category in error_categories:
        if error is not None:
            if category in str(error):
                return category
    return None

pyflakes_error_categories = ['undefined name', 'assigned to but never used', 'imported but unused']
python_codes['pyflakes_error_category'] = python_codes['pyflakes_error'].apply(get_error_category, error_categories=pyflakes_error_categories)
python_codes.loc[python_codes.is_valid_python_with_compile == False, 'pyflakes_error_category'] = 'Invalid Syntax'
python_codes.loc[(python_codes.is_valid_python_with_pyflakes == False) & (python_codes.pyflakes_error_category.isnull()), 'pyflakes_error_category'] = 'Other'

python_codes['pyflakes_error_category'].value_counts()


In [ ]:
ruff_error_categories = ["{None}", "{'F821'}", "{'F822'}", "{'F823'}"]
python_codes['ruff_error_category'] = python_codes['ruff_error'].apply(get_error_category, error_categories=ruff_error_categories)
python_codes['ruff_error_category'].value_counts()

In [ ]:
python_codes[(python_codes.is_valid_python_with_ruff == True) & (python_codes.pyflakes_error_category == 'undefined name')][['code', 'ruff_error', 'pyflakes_error']].head()

In [5]:
# python_codes.to_csv('/mnt/foundation-shared/nina_xu_gretel_ai/datasets/python_codes_with_checks_0927.csv', index=False)
python_codes = pd.read_csv('/mnt/foundation-shared/nina_xu_gretel_ai/datasets/python_codes_with_checks_0927.csv')


In [172]:
# python_codes[python_codes.is_valid_python_with_pyflakes == False][['code', 'pyflakes_error', 'is_valid_python_with_compile']].head(30)

In [ ]:
# compile errors
ind = 15
ind = 115
# pyflakes errors
ind = 2 # imported but unused
ind = 69 # assigned to but never used
ind = 36 # undefined name
# mypy errors
ind = 576 # missing positional argument
ind = 743 # unsupported operand types
ind = 545 # has no attribute X
# incomplete code
ind = 261

ind = 509
print(python_codes.prompt[ind])
print('----------\n')
print(python_codes.code[ind])

In [ ]:
python_codes.error_category[(python_codes.is_valid_python_with_mypy == False)].value_counts()

In [ ]:
python_codes[(python_codes.is_valid_python_with_mypy == False)][['mypy_error', 'pyflakes_error_category']].head(30)

In [ ]:
# python_codes[python_codes.pyflakes_error_category == 'undefined name'][['pyflakes_error', 'mypy_error']]

In [ ]:
python_codes['incomplete_code']= python_codes.code.apply(lambda x: '# ...' in x)
print(python_codes.incomplete_code.value_counts())
python_codes[python_codes.incomplete_code == True][['code', 'pyflakes_error']]